<a href="https://colab.research.google.com/github/fernandogarade/CEIA_09_NLP/blob/main/Desaf%C3%ADo_Custom_embeddings_con_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Descargar la carpeta de dataset
import os
import gdown
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('simpsons_dataset.zip', os.F_OK) is False:
        url = 'https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download'
        output = 'songs_dataset.zip'
        gdown.download(url, output, quiet=False)
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

Downloading...
From: https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download
To: /content/songs_dataset.zip
100%|██████████| 2.09M/2.09M [00:00<00:00, 128MB/s]


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['r-kelly.txt',
 'nirvana.txt',
 'Lil_Wayne.txt',
 'adele.txt',
 'nursery_rhymes.txt',
 'bjork.txt',
 'dickinson.txt',
 'cake.txt',
 'paul-simon.txt',
 'dr-seuss.txt',
 'beatles.txt',
 'amy-winehouse.txt',
 'notorious-big.txt',
 'kanye-west.txt',
 'lin-manuel-miranda.txt',
 'missy-elliott.txt',
 'disney.txt',
 'bieber.txt',
 'lorde.txt',
 'bob-dylan.txt',
 'rihanna.txt',
 'bruno-mars.txt',
 'nickelback.txt',
 'notorious_big.txt',
 'joni-mitchell.txt',
 'al-green.txt',
 'jimi-hendrix.txt',
 'eminem.txt',
 'janisjoplin.txt',
 'nicki-minaj.txt',
 'michael-jackson.txt',
 'dolly-parton.txt',
 'lil-wayne.txt',
 'prince.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'leonard-cohen.txt',
 'alicia-keys.txt',
 'lady-gaga.txt',
 'blink-182.txt',
 'johnny-cash.txt',
 'drake.txt',
 'Kanye_West.txt',
 'britney-spears.txt',
 'bob-marley.txt',
 'patti-smith.txt',
 'ludacris.txt',
 'bruce-springsteen.txt',
 'kanye.txt']

In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
# Dataset Eminem
df_eminem = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)

# Dataset Rihanna
df_rihanna = pd.read_csv('songs_dataset/rihanna.txt', sep='/n', header=None)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [6]:
df_eminem.head()

,0
0,"Look, I was gonna go easy on you and not to hu..."
1,But I'm only going to get this one chance
2,"Something's wrong, I can feel it (Six minutes,..."
3,"Just a feeling I've got, like something's abou..."
4,"If that means, what I think it means, we're in..."


In [7]:
df_rihanna.head()

,0
0,Ghost in the mirror
1,"I knew your face once, but now it's unclear"
2,And I can't feel my body now
3,I'm separate from here and now A drug and a dream
4,"We lost connection, oh come back to me"


In [8]:
print("Cantidad de documentos eminem:", df_eminem.shape[0])
print("Cantidad de documentos rihanna:", df_rihanna.shape[0])

Cantidad de documentos eminem: 6812
Cantidad de documentos rihanna: 3895


### 1 - Preprocesamiento

In [9]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens_eminem = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df_eminem[:None].iterrows():
    sentence_tokens_eminem.append(text_to_word_sequence(row[0]))

sentence_tokens_rihanna = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df_rihanna[:None].iterrows():
    sentence_tokens_rihanna.append(text_to_word_sequence(row[0]))

In [11]:
# Demos un vistazo
sentence_tokens_eminem[:2]

[['look',
  'i',
  'was',
  'gonna',
  'go',
  'easy',
  'on',
  'you',
  'and',
  'not',
  'to',
  'hurt',
  'your',
  'feelings'],
 ['but', "i'm", 'only', 'going', 'to', 'get', 'this', 'one', 'chance']]

In [10]:
# Demos un vistazo
sentence_tokens_rihanna[:2]

[['ghost', 'in', 'the', 'mirror'],
 ['i', 'knew', 'your', 'face', 'once', 'but', 'now', "it's", 'unclear']]

### 2 - Crear los vectores (word2vec)

In [12]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [13]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo CBOW
w2v_model_eminem = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

w2v_model_rihanna = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [14]:
# Buildear el vocabulario con los tokens
w2v_model_eminem.build_vocab(sentence_tokens_eminem)

w2v_model_rihanna.build_vocab(sentence_tokens_rihanna)

In [15]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus de eminem:", w2v_model_eminem.corpus_count)
print("Cantidad de docs en el corpus de rihanna:", w2v_model_rihanna.corpus_count)

Cantidad de docs en el corpus de eminem: 6812
Cantidad de docs en el corpus de rihanna: 3895


In [16]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus de eminem:", len(w2v_model_eminem.wv.vocab))
print("Cantidad de words distintas en el corpus de rihanna:", len(w2v_model_rihanna.wv.vocab))

Cantidad de words distintas en el corpus de eminem: 1347
Cantidad de words distintas en el corpus de rihanna: 660


### 3 - Entrenar el modelo generador

In [20]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_eminem.train(sentence_tokens_eminem,
                 total_examples=w2v_model_eminem.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 134750.390625
Loss after epoch 1: 130926.296875
Loss after epoch 2: 127738.59375
Loss after epoch 3: 124569.0
Loss after epoch 4: 118723.59375
Loss after epoch 5: 114442.5625
Loss after epoch 6: 112670.5
Loss after epoch 7: 109887.9375
Loss after epoch 8: 105901.0
Loss after epoch 9: 98632.5
Loss after epoch 10: 97646.875
Loss after epoch 11: 95614.0
Loss after epoch 12: 93633.25
Loss after epoch 13: 92742.25
Loss after epoch 14: 92211.625
Loss after epoch 15: 89832.125
Loss after epoch 16: 89784.625
Loss after epoch 17: 88360.875
Loss after epoch 18: 86726.5
Loss after epoch 19: 86602.125
Loss after epoch 20: 80801.125
Loss after epoch 21: 79853.75
Loss after epoch 22: 78972.0
Loss after epoch 23: 78853.0
Loss after epoch 24: 77735.25
Loss after epoch 25: 77614.5
Loss after epoch 26: 76597.0
Loss after epoch 27: 76503.25
Loss after epoch 28: 75374.0
Loss after epoch 29: 75471.0
Loss after epoch 30: 75120.5
Loss after epoch 31: 74653.75
Loss after epoch 32: 74461.5


(3930601, 6483800)

In [19]:
w2v_model_rihanna.train(sentence_tokens_rihanna,
                 total_examples=w2v_model_rihanna.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 56060.453125
Loss after epoch 1: 53750.625
Loss after epoch 2: 52196.140625
Loss after epoch 3: 50403.65625
Loss after epoch 4: 49001.875
Loss after epoch 5: 47493.4375
Loss after epoch 6: 46506.0625
Loss after epoch 7: 45209.40625
Loss after epoch 8: 43829.96875
Loss after epoch 9: 42984.6875
Loss after epoch 10: 42257.0
Loss after epoch 11: 40406.625
Loss after epoch 12: 39620.875
Loss after epoch 13: 38887.3125
Loss after epoch 14: 38231.1875
Loss after epoch 15: 37671.8125
Loss after epoch 16: 37992.4375
Loss after epoch 17: 37001.6875
Loss after epoch 18: 37000.625
Loss after epoch 19: 36566.1875
Loss after epoch 20: 36056.3125
Loss after epoch 21: 35719.3125
Loss after epoch 22: 35371.0625
Loss after epoch 23: 35423.8125
Loss after epoch 24: 34779.3125
Loss after epoch 25: 33693.25
Loss after epoch 26: 33354.5
Loss after epoch 27: 32788.25
Loss after epoch 28: 33270.375
Loss after epoch 29: 32648.875
Loss after epoch 30: 32883.875
Loss after epoch 31: 32422.62

(1786806, 3017900)

### 4 - Ensayar

In [28]:
# Palabras que MÁS se relacionan con...:
w2v_model_eminem.wv.most_similar(positive=["you"], topn=10)

[('i', 0.5925841927528381),
 ('they', 0.48412638902664185),
 ('me', 0.46879324316978455),
 ('it', 0.4563320577144623),
 ('just', 0.3848724365234375),
 ('we', 0.3765586316585541),
 ('that', 0.3650105893611908),
 ("don't", 0.3496406674385071),
 ("y'all", 0.34360986948013306),
 ('but', 0.32276538014411926)]

In [27]:
# Palabras que MÁS se relacionan con...:
w2v_model_rihanna.wv.most_similar(positive=["you"], topn=10)

[('i', 0.5055983066558838),
 ('me', 0.37097030878067017),
 ('to', 0.33557185530662537),
 ('it', 0.29936838150024414),
 ('they', 0.29156580567359924),
 ('him', 0.28220003843307495),
 ('some', 0.2717239260673523),
 ('so', 0.2644213140010834),
 ('ever', 0.26075857877731323),
 ('touch', 0.25831353664398193)]

Vemos que utilizando cualquiera de los dos corpus, las palabras que más se realcionan con 'you' son otros pronombres personales como i, they o me.

In [29]:
# Palabras que MÁS se relacionan con...:
w2v_model_eminem.wv.most_similar(positive=["love"], topn=10)

[('hate', 0.3057187497615814),
 ('only', 0.2860211730003357),
 ('swear', 0.2568838894367218),
 ('need', 0.24828752875328064),
 ('forget', 0.23617541790008545),
 ('favors', 0.2309073656797409),
 ('longer', 0.23083369433879852),
 ('drive', 0.22008895874023438),
 ('hear', 0.21293339133262634),
 ('ma', 0.21262745559215546)]

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model_rihanna.wv.most_similar(positive=["love"], topn=10)

[('mine', 0.36859679222106934),
 ('baby', 0.3198685646057129),
 ('hate', 0.31841856241226196),
 ('need', 0.30098867416381836),
 ('lie', 0.29890236258506775),
 ('used', 0.2822553813457489),
 ('stupid', 0.27180230617523193),
 ('soul', 0.2703780233860016),
 ('eye', 0.26519453525543213),
 ('talk', 0.2501453459262848)]

Sin embargo, al analizar las palabras que se encuentran más cercanas de 'love' cuando se entrenan los modelos con uno y otro corpus las palabras más próximas son muy distintas (con excepción de hate que aparece en ambos TOP 10)

### 5 - Visualizar agrupación de vectores

In [34]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [36]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model_eminem)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [37]:
x_vals, y_vals, labels = reduce_dimensions(w2v_model_rihanna)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab